## Imports

In [2]:
# Imports
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json
from tqdm import tqdm
import math
from scipy.stats import wasserstein_distance

## Declares

In [3]:
# MAC OSX
WD = "/Users/alex/Documents/BUSTED_ModelTest"

# Dataset tag (User defined) ---
tag = "14-datasets"

# Additional declares
#BUSTEDS_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS")
#BUSTEDS_MH_DIR = os.path.join(WD, "analysis", tag, "BUSTEDS-MH")
RESULTS_DIR = os.path.join(WD, "analysis")

# Create tables folder
OUTPUT_CSV = os.path.join(WD, "tables", "Table_" + tag + "_results.csv")

ER_Threshold = 5

In [10]:
def read_json(filename):
    #print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        print("# -- Error -- file is empty:", filename)
        return []
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method

#define function to calculate cv
#cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
cv = lambda x: np.std(x) / np.mean(x)

pctchg = lambda a, b: (a / b) * 100

In [23]:
def process(FILES, fileending, method):
    df_dict = {}
    for item in tqdm(FILES):
        basename = ""
        if fileending in os.path.basename(item):
            basename = os.path.basename(item).replace(fileending, "")
        #end if
        
        for fext in [".phy", ".fasta", ".nex", "-align-dna.fas", "-Aligned-DNA.fas"]:
            basename = basename.replace(fext, "")
        #end for
        
        json_data = read_json(item) # Read json

        if json_data == []:
            continue # Empty file
        #end if
        df_dict[basename] = {"Method": method}
        df_dict[basename].update({"Sequences": json_data["input"]["number of sequences"]})
        df_dict[basename].update({"Codons": json_data["input"]["number of sites"]})
        df_dict[basename].update({"LRT p-value": json_data["test results"]["p-value"]})

        # cAIC
        df_dict[basename].update({"cAIC": json_data["fits"]["Unconstrained model"]["AIC-c"]})
        
        # Omegas
        A = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
        B = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
        C = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
        df_dict[basename].update({"CV(omega)": cv([A, B, C])})

        try:
            D = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
            E = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
            F = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
            df_dict[basename].update({"CV(alpha)": cv([D, E, F])})
        except:
            pass
        #end try
        
        #Omegas and proportions
        data = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]
        w1 = data["0"]["omega"]
        p1 = data["0"]["proportion"]
        w2 = data["1"]["omega"]
        p2 = data["1"]["proportion"]
        w3 = data["2"]["omega"]
        p3 = data["2"]["proportion"]
        df_dict[basename].update({"w1": w1, "p1": p1})
        df_dict[basename].update({"w2": w2, "p2": p2})
        df_dict[basename].update({"w3": w3, "p3": p3})

        # SRV rates and proportions
        if method == "BUSTEDS-MH" or method == "BUSTEDS":
            data = json_data["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]
            s1 = data["0"]["rate"]
            s_p1 = data["0"]["proportion"]
            s2 = data["1"]["rate"]
            s_p2 = data["1"]["proportion"]
            s3 = data["2"]["rate"]
            s_p3 = data["2"]["proportion"]
            df_dict[basename].update({"SRV1": s1, "SRV_p1": s_p1})
            df_dict[basename].update({"SRV2": s2, "SRV_p2": s_p2})
            df_dict[basename].update({"SRV3": s3, "SRV_p3": s_p3})
        
        
        # DH rate, TH rate, TH_SI rate
        if method == "BUSTEDS-MH":
            df_dict[basename].update({"DH_Rate": float(json_data["fits"]["Unconstrained model"]
                                  ["rate at which 2 nucleotides are changed instantly within a single codon"])})
            df_dict[basename].update({"TH_Rate": float(json_data["fits"]["Unconstrained model"]
                                  ["rate at which 3 nucleotides are changed instantly within a single codon"])})
            df_dict[basename].update({"TH_Rate_SI": float(json_data["fits"]["Unconstrained model"]
                                  ["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"])})

        
        # ER Sites
        ER_SITES = []
        ER_df_dict = {}

        if "constrained" in json_data["Evidence Ratios"].keys():
            for site, val in enumerate(json_data["Evidence Ratios"]["constrained"][0]):
                if val > ER_Threshold:
                    ER_SITES.append(str(site + 1))
                    ER_df_dict[site + 1] = {"ER": val}
                #end if
            #end for
            df_dict[basename].update({"NUM_ER_SITES":  len(ER_df_dict.keys())})
            x = ER_df_dict.keys()
            x = [str(x) for x in x]
            df_dict[basename].update({"ER_SITES":  "|".join(x)})
        #end if   
    # End for
    df = pd.DataFrame.from_dict(df_dict, orient="index")
    df = df.reset_index()
    df.index += 1
    df.rename(columns={'index': 'Gene'}, inplace = True)
    return df
#end method

## Look over results

In [24]:
BUSTEDS_RESULTS = [os.path.join(RESULTS_DIR, file.name) for file in os.scandir(RESULTS_DIR) if file.name.endswith(".BUSTEDS.json")]
BUSTEDS_MH_RESULTS = [os.path.join(RESULTS_DIR, file.name) for file in os.scandir(RESULTS_DIR) if file.name.endswith(".BUSTEDS-MH.json")]
BUSTED_RESULTS = [os.path.join(RESULTS_DIR, file.name) for file in os.scandir(RESULTS_DIR) if file.name.endswith(".BUSTED.json")]
BUSTED_MH_RESULTS = [os.path.join(RESULTS_DIR, file.name) for file in os.scandir(RESULTS_DIR) if file.name.endswith(".BUSTED-MH.json")]

print("# Number of BUSTEDS results:", len(BUSTEDS_RESULTS))
print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_RESULTS))
print("# Number of BUSTED results:", len(BUSTED_RESULTS))
print("# Number of BUSTED-MH results:", len(BUSTED_MH_RESULTS))

#print("# Number of SLAC results:", len(SLAC_DIR_FILES))

# Number of BUSTEDS results: 14
# Number of BUSTEDS-MH results: 14
# Number of BUSTED results: 14
# Number of BUSTED-MH results: 14


In [25]:

print("# Processing BUSTED[S] files")
df_BUSTEDS = process(BUSTEDS_RESULTS , ".BUSTEDS.json", "BUSTEDS")

print("# Processing BUSTED[S]-MH files")
df_BUSTEDS_MH = process(BUSTEDS_MH_RESULTS , ".BUSTEDS-MH.json", "BUSTEDS-MH")

print("# Processing BUSTED files")
df_BUSTED = process(BUSTED_RESULTS , ".BUSTED.json", "BUSTED")

print("# Processing BUSTED-MH files")
df_BUSTED_MH = process(BUSTEDS_MH_RESULTS, ".BUSTED-MH.json", "BUSTED-MH")


# Processing BUSTED[S] files


100%|██████████| 14/14 [00:00<00:00, 360.66it/s]


# Processing BUSTED[S]-MH files


100%|██████████| 14/14 [00:00<00:00, 372.59it/s]


# Processing BUSTED files


100%|██████████| 14/14 [00:00<00:00, 476.45it/s]


# Processing BUSTED-MH files


100%|██████████| 14/14 [00:00<00:00, 416.56it/s]


In [29]:
df_BUSTEDS

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),CV(alpha),w1,p1,...,w3,p3,SRV1,SRV_p1,SRV2,SRV_p2,SRV3,SRV_p3,NUM_ER_SITES,ER_SITES
1,HIVvif,BUSTEDS,29,192,2.383380e-02,6913.790452,1.411915,0.827416,0.716583,0.990546,...,1582.603868,0.000491,0.295969,0.545601,1.158556,0.325050,3.571189,0.129349,1.0,6
2,lysozyme,BUSTEDS,19,130,5.000000e-01,2145.579372,0.368839,0.964383,0.426424,0.809564,...,1.011802,0.000000,0.000000,0.453174,1.309679,0.452809,4.328611,0.094018,NaN,NaN
3,COXI,BUSTEDS,21,510,5.000000e-01,24288.101212,1.381566,1.330024,0.000977,0.673833,...,1.000000,0.011274,0.033245,0.018959,0.575680,0.950062,14.604620,0.030979,NaN,NaN
4,flavNS5,BUSTEDS,18,342,4.826393e-01,18530.532597,1.352523,1.215819,0.008133,0.796769,...,1.180745,0.018878,0.120672,0.187685,0.524341,0.714772,6.177431,0.097543,0.0,
5,bglobin,BUSTEDS,17,144,8.467278e-06,7415.047518,1.304447,1.087483,0.035399,0.828310,...,16.225672,0.020041,0.339402,0.386773,1.056381,0.579178,7.544826,0.034049,10.0,10|11|42|48|50|54|74|110|116|124
6,ENCenv,BUSTEDS,23,500,5.000000e-01,13699.103102,1.254676,0.893186,0.031310,0.160025,...,1.044311,0.000000,0.403572,0.305010,1.134033,0.669239,4.581135,0.025751,NaN,NaN
7,yokoyama.rh1.cds.mod.1-990,BUSTEDS,38,330,1.146392e-06,25924.019274,1.371073,1.023342,0.000000,0.609172,...,6.588823,0.012658,0.381562,0.592894,1.121660,0.348432,6.526708,0.058675,30.0,13|14|16|19|33|35|36|42|46|93|96|107|108|122|1...
8,HIV_RT,BUSTEDS,476,335,1.535105e-12,52026.769254,1.381185,0.833936,0.122886,0.962661,...,70.821117,0.000687,0.402784,0.515996,1.209478,0.412551,4.103288,0.071453,17.0,48|64|69|75|104|122|138|151|162|163|181|188|21...
9,adh,BUSTEDS,23,254,1.815529e-03,9354.009509,1.383019,0.574009,0.020131,0.173311,...,4.102014,0.025319,0.605146,0.268544,0.743412,0.506222,2.047467,0.225235,18.0,4|6|35|39|46|49|68|69|133|134|163|165|166|170|...
10,lysin,BUSTEDS,25,134,0.000000e+00,8761.171230,1.296509,0.827026,0.000000,0.376865,...,16.745293,0.077705,0.171040,0.365657,1.134931,0.522246,3.075401,0.112098,36.0,3|4|6|7|10|12|14|15|16|27|30|36|37|41|44|45|63...


In [30]:
df_BUSTEDS_MH

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),CV(alpha),w1,p1,...,SRV_p1,SRV2,SRV_p2,SRV3,SRV_p3,DH_Rate,TH_Rate,TH_Rate_SI,NUM_ER_SITES,ER_SITES
1,InfluenzaA,BUSTEDS-MH,349,329,0.500000,23230.195553,0.474401,0.877723,3.619313e-01,0.000000,...,0.648516,1.673160,0.319125,5.883673,0.032359,0.062803,0.014882,0.000000,NaN,NaN
2,COXI,BUSTEDS-MH,21,510,0.500000,24295.058286,0.701211,1.306716,5.584007e-03,0.989112,...,0.020285,0.618153,0.945608,12.158473,0.034107,0.000000,0.000000,4.371442,NaN,NaN
3,flavNS5,BUSTEDS-MH,18,342,0.500000,18488.578696,1.380767,1.079050,6.219587e-08,0.000000,...,0.214842,0.627566,0.662687,4.399485,0.122471,0.362455,0.907991,2.395013,NaN,NaN
4,ENCenv,BUSTEDS-MH,23,500,0.500000,13705.123029,1.282742,0.891511,4.902972e-02,0.937270,...,0.309724,1.137939,0.664743,4.587889,0.025533,0.012449,0.000000,0.000000,NaN,NaN
5,adh,BUSTEDS-MH,23,254,0.019489,9359.111669,1.383861,0.552316,1.615062e-02,0.107078,...,0.673245,1.413463,0.251789,2.808413,0.074966,0.000000,0.033096,3.364916,15.0,6|35|39|49|69|133|134|163|165|166|170|197|216|...
6,HIVvif,BUSTEDS-MH,29,192,0.500000,6913.086831,0.252332,0.820836,5.849111e-01,0.119606,...,0.541747,1.152659,0.326605,3.507036,0.131647,0.001566,0.162595,0.000000,NaN,NaN
7,camelid,BUSTEDS-MH,212,96,0.006905,33668.701415,0.922319,0.804136,4.813634e-01,0.776627,...,0.306967,0.829094,0.474061,2.454190,0.218972,0.173423,0.016813,0.000000,26.0,1|11|14|23|25|29|31|32|33|40|50|51|52|53|54|55...
8,VOI_SPIKE,BUSTEDS-MH,180,1284,0.000182,17811.389274,1.178151,1.050279,2.686437e-01,0.487225,...,0.906070,5.607647,0.082028,26.516610,0.011902,0.010758,0.000000,0.000000,44.0,5|9|18|19|20|67|70|80|95|211|213|219|220|221|2...
9,lysozyme,BUSTEDS-MH,19,130,0.500000,2151.860776,1.284331,0.964502,3.780395e-01,0.650430,...,0.453522,1.313539,0.453336,4.343110,0.093142,0.000000,0.000000,0.000000,NaN,NaN
10,HIV_RT,BUSTEDS-MH,476,335,0.001235,52037.043272,1.407171,0.826657,0.000000e+00,0.018400,...,0.510042,1.194852,0.414642,3.987729,0.075316,0.039729,0.000000,0.000000,12.0,48|64|69|75|122|151|162|181|188|215|228|245


In [33]:
df_BUSTED

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),w1,p1,w2,p2,w3,p3,NUM_ER_SITES,ER_SITES
1,yokoyama.rh1.cds.mod.1-990,BUSTED,38,330,9.992007e-16,26416.479399,1.401356,0.000000,0.054603,0.047631,0.920082,7.798889,0.025314,69.0,7|8|13|14|16|19|26|33|35|37|39|42|46|50|54|81|...
2,lysozyme,BUSTED,19,130,3.769792e-01,2165.851278,1.132226,0.188525,0.865726,0.281840,0.000000,3.062285,0.134274,5.0,37|41|50|101|114
3,flavNS5,BUSTED,18,342,4.220719e-01,18789.833822,1.390279,0.005805,0.949834,0.007204,0.005898,1.139867,0.044268,0.0,
4,camelid,BUSTED,212,96,0.000000e+00,35140.302106,1.277873,0.496019,0.915425,1.000000,0.033814,21.733517,0.050761,41.0,1|11|14|23|24|25|27|28|29|30|31|32|33|35|40|44...
5,InfluenzaA,BUSTED,349,329,6.970205e-09,23863.727926,1.330142,0.347174,0.986079,1.000000,0.009692,32.560865,0.004229,25.0,15|53|121|133|135|137|138|145|156|157|159|172|...
6,HIV_RT,BUSTED,476,335,0.000000e+00,53749.794833,1.350796,0.079227,0.935271,1.000000,0.060956,34.814021,0.003773,46.0,35|36|39|48|49|64|65|69|75|83|98|100|103|104|1...
7,HIVvif,BUSTED,29,192,2.005439e-04,7101.856673,1.178539,0.330011,0.630743,0.422008,0.285168,6.013574,0.084090,32.0,6|19|22|31|33|36|37|39|41|47|48|51|63|66|91|92...
8,lysin,BUSTED,25,134,0.000000e+00,8921.661582,1.297757,0.000000,0.583581,1.000000,0.311474,16.938709,0.104945,61.0,2|3|4|6|7|8|9|10|11|12|14|15|27|30|32|33|36|37...
9,adh,BUSTED,23,254,2.624235e-04,9371.692413,1.085644,0.033896,0.968429,1.000000,0.009224,5.330541,0.022347,24.0,2|4|35|39|49|57|68|69|81|98|133|134|163|165|16...
10,bglobin,BUSTED,17,144,1.161271e-11,7452.740235,1.313216,0.044568,0.853287,1.000000,0.119840,20.657268,0.026872,18.0,3|7|10|11|18|42|48|49|50|54|67|74|85|110|114|1...


In [34]:
df_BUSTED_MH

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),CV(alpha),w1,p1,w2,p2,w3,p3,NUM_ER_SITES,ER_SITES
1,,BUSTED-MH,25,134,0.000041,8774.540564,1.414214,0.687814,0,0.23604,0,0.36706,3.098741,0.396901,41,4|6|7|9|10|11|12|14|32|33|36|37|40|41|44|45|47...


## Concatenate tables


In [78]:
result = pd.concat([df_MH, df])
result = result.fillna("")
result = result.sort_values(by=["Gene", "Method"], ascending=True)
result = result.reset_index(drop=True)
result.index += 1
result

,Gene,Method,Sequences,Codons,LRT p-value,cAIC,CV(omega),CV(alpha),w1,p1,...,TH_Rate_SI,NUM_ER_SITES,ER_SITES,WD_unweighted(omega),WD_weighted(omega),WD_unweighted(srv),WD_weighted(srv),ΔcAIC,RelativeSupport,ER_Sites_Intersection
1,Anguilliformes-ATP6,BUSTEDS,22,229,0.314174,16081.542634,1.322909,1.154666,0.010975,0.818510,...,,0.0,,,,,,,,
2,Anguilliformes-ATP6,BUSTEDS-MH,22,229,0.500000,16064.881575,1.273617,1.059347,0.023284,1.000000,...,2.747741,,,0.290244,0.034365,0.16765,0.185132,16.661059,0.000241,
3,Anguilliformes-ATP8,BUSTEDS,22,63,0.480028,4495.920303,1.289913,1.021358,0.000000,0.127324,...,,0.0,,,,,,2.615669,0.270405,
4,Anguilliformes-ATP8,BUSTEDS-MH,22,63,0.500000,4498.535972,1.176105,0.981995,0.077865,1.000000,...,317.922626,,,0.056851,0.048348,0.035495,0.056818,,,
5,Anguilliformes-COX1,BUSTEDS,22,545,0.145802,28936.365521,1.413745,1.364689,0.001340,0.845038,...,,1.0,265,,,,,11.562144,0.003085,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,zeiformes-nd4l,BUSTEDS-MH,6,99,0.500000,1846.402168,1.389156,0.657191,0.000551,0.000000,...,0.225298,,,0.007399,0.000396,2.415088,0.15144,,,
867,zeiformes-nd5,BUSTEDS,6,613,0.471917,12483.963288,1.273600,1.233760,0.043067,0.362322,...,,0.0,,,,,,,,
868,zeiformes-nd5,BUSTEDS-MH,6,613,0.500000,12480.883526,1.222350,1.119733,0.047367,0.000000,...,0.0,,,0.07573,0.04094,1.312711,0.283658,3.079761,0.214407,
869,zeiformes-nd6,BUSTEDS,6,174,0.031906,3500.519163,1.413401,1.080664,0.000000,0.259944,...,,2.0,107|124,,,,,5.442524,0.065792,


In [79]:
result.columns

Index(['Gene', 'Method', 'Sequences', 'Codons', 'LRT p-value', 'cAIC',
       'CV(omega)', 'CV(alpha)', 'w1', 'p1', 'w2', 'p2', 'w3', 'p3', 'SRV1',
       'SRV_p1', 'SRV2', 'SRV_p2', 'SRV3', 'SRV_p3', 'DH_Rate', 'TH_Rate',
       'TH_Rate_SI', 'NUM_ER_SITES', 'ER_SITES', 'WD_unweighted(omega)',
       'WD_weighted(omega)', 'WD_unweighted(srv)', 'WD_weighted(srv)', 'ΔcAIC',
       'RelativeSupport', 'ER_Sites_Intersection'],
      dtype='object')

## Save table

In [80]:
print("Saving results to:", OUTPUT_CSV)
result.to_csv(OUTPUT_CSV, index=False)

Saving results to: /Users/alex/Documents/BUSTEDS-MH/tables/Table_Empirical_mtDNAVertebrate_results.csv


## End of file